In [16]:
from mysklearn.mypytable import MyPyTable
import mysklearn.myutils as myutils
from mysklearn.myclassifiers import MyDummyClassifier


table = MyPyTable().load_from_file("input_data/tracks.csv")

def categorize_popularity(p):
    p = int(p)
    if p <= 33:
        return "Low"
    elif p <= 66:
        return "Medium"
    else:
        return "High"

pop_idx = table.column_names.index("popularity")
y = [categorize_popularity(row[pop_idx]) for row in table.data]

explicit_idx = table.column_names.index("explicit")
duration_idx = table.column_names.index("duration_ms")
dance_idx = table.column_names.index("danceability")
energy_idx = table.column_names.index("energy")
tempo_idx = table.column_names.index("tempo")
loudness_idx = table.column_names.index("loudness")

X = []
for row in table.data:
    X.append([
        row[explicit_idx],
        row[duration_idx],
        row[dance_idx],
        row[energy_idx],
        row[tempo_idx],
        row[loudness_idx]
    ])

dummyClf = MyDummyClassifier()
dummyClf.fit(X, y)

# Predict an example song
example_song = [
    1,         # explicit
    150000,    # duration_ms
    0.25,      # danceability
    0.50,      # energy
    -8.0       # loudness
]

print("Prediction:", dummyClf.predict([example_song]))

Prediction: ['Low']


In [17]:
import mysklearn.myevaluation as myevaluation

dummy_acc, dummy_err, dummy_tp, dummy_p = myutils.cross_val_predict(X, y, MyDummyClassifier)

print(f"Dummy acc: {dummy_acc:.2f} Dummy err: {dummy_err:.2f}")

dummy_prec = myevaluation.binary_precision_score(dummy_tp, dummy_p)

print(f"Dummy precision: {dummy_prec:.2f}")

dummy_recall = myevaluation.binary_recall_score(dummy_tp, dummy_p)

print(f"Dummy recall: {dummy_recall:.2f}")

dummy_f1 = myevaluation.binary_f1_score(dummy_tp, dummy_p)

print(f"Dummy f1: {dummy_f1:.2f}")

Dummy acc: 0.62 Dummy err: 0.38
Dummy precision: 0.00
Dummy recall: 0.00
Dummy f1: 0.00
